In [1]:
import os
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from tqdm import tqdm
import clean_data_v3 as clean_data

# Aktuelles Arbeitsverzeichnis abrufen
current_dir = os.getcwd().replace("\\", "/")

# Trainings- und Testdaten einlesen
df_train = pd.read_csv(current_dir + "/data/train.csv")
df_test = pd.read_csv(current_dir + "/data/test.csv")

# Daten bereinigen
df_train = clean_data.clean_data(df_train, is_train=True)
df_test = clean_data.clean_data(df_test, is_train=False)

# Überprüfen, ob 'efs' nach dem Cleaning vorhanden ist
if 'efs' not in df_train:
    raise ValueError("Fehlende Zielvariable ('efs') nach dem Cleaning!")

# Features definieren (nur numerische Spalten außer 'efs')
X_train = df_train.drop(columns=['efs', 'efs_time'], errors='ignore')
y_train = df_train['efs'].values


X_test = df_test.drop(columns=['efs', 'efs_time'], errors='ignore')


# XGBRegressor initialisieren
xgb_model = XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=5,
    random_state=42
)

# Training mit Fortschrittsbalken
with tqdm(total=1, desc="Training des Modells") as pbar:
    xgb_model.fit(X_train, y_train)
    pbar.update(1)

# Vorhersagen für Testdaten (Risikoscore für efs)
risk_scores = xgb_model.predict(X_test)

# Ergebnisse speichern
results = pd.DataFrame({'ID': df_test.index, 'risk_score': risk_scores})
results.head(5)


ℹ️ Kategorische Spalte 'dri_score': Fehlende Werte mit 'Unknown' auffüllen.
ℹ️ Kategorische Spalte 'psych_disturb': Fehlende Werte mit 'Unknown' auffüllen.
ℹ️ Kategorische Spalte 'cyto_score': Fehlende Werte mit 'Unknown' auffüllen.
ℹ️ Kategorische Spalte 'diabetes': Fehlende Werte mit 'Unknown' auffüllen.
ℹ️ Numerische Spalte 'hla_match_c_high': Fehlende Werte mit Median auffüllen.
ℹ️ Numerische Spalte 'hla_high_res_8': Fehlende Werte mit Median auffüllen.
ℹ️ Kategorische Spalte 'arrhythmia': Fehlende Werte mit 'Unknown' auffüllen.
ℹ️ Numerische Spalte 'hla_low_res_6': Fehlende Werte mit Median auffüllen.
ℹ️ Kategorische Spalte 'vent_hist': Fehlende Werte mit 'Unknown' auffüllen.
ℹ️ Kategorische Spalte 'renal_issue': Fehlende Werte mit 'Unknown' auffüllen.
ℹ️ Kategorische Spalte 'pulm_severe': Fehlende Werte mit 'Unknown' auffüllen.
ℹ️ Numerische Spalte 'hla_high_res_6': Fehlende Werte mit Median auffüllen.
ℹ️ Kategorische Spalte 'cmv_status': Fehlende Werte mit 'Unknown' auffüllen.
ℹ

/home/fa23b_eric/notebooks/My Projects/Mittelstufenprojekt/clean_data_v3.py:40: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna("Unknown", inplace=True)
/home/fa23b_eric/notebooks/My Projects/Mittelstufenprojekt/clean_data_v3.py:36: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as

Numerische Spalten mit NaN-Werten: []
⚠️ Problematische Spalten: ['dri_score_High - TED AML case <missing cytogenetics', 'dri_score_Intermediate - TED AML case <missing cytogenetics', 'dri_score_Missing disease status', 'dri_score_N/A - disease not classifiable', 'dri_score_N/A - non-malignant indication', 'dri_score_N/A - pediatric', 'dri_score_TBD cytogenetics', 'dri_score_Very high', 'cyto_score_Not tested', 'tbi_status_TBI + Cy +- Other', 'tbi_status_TBI +- Other, -cGy, fractionated', 'tbi_status_TBI +- Other, -cGy, single', 'tbi_status_TBI +- Other, -cGy, unknown dose', 'tbi_status_TBI +- Other, <=cGy', 'tbi_status_TBI +- Other, >cGy', 'tbi_status_TBI +- Other, unknown dose', 'graft_type_Peripheral blood', 'prim_disease_hct_Other acute leukemia', 'prim_disease_hct_Other leukemia', 'prim_disease_hct_Solid tumor', 'cyto_score_detail_Not tested', 'conditioning_intensity_N/A, F(pre-TED) not submitted', 'conditioning_intensity_No drugs reported', 'ethnicity_Non-resident of the U.S.', '

Training des Modells:   0%|                               | 0/1 [00:00<?, ?it/s]


AttributeError: 'DataFrame' object has no attribute 'dtype'

In [8]:
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("X_train columns:", X_train.columns)
print("X_test columns:", X_test.columns)
print("X_train dtypes:", X_train.dtypes)
print("X_test dtypes:", X_test.dtypes)print("X_train columns:", X_train.columns)
print("X_test columns:", X_test.columns)
print("X_train dtypes:", X_train.dtypes)
print("X_test dtypes:", X_test.dtypes)


X_train shape: (28800, 163)
X_test shape: (3, 44)
X_train columns: Index(['ID', 'hla_match_c_high', 'hla_high_res_8', 'hla_low_res_6',
       'hla_high_res_6', 'hla_high_res_10', 'hla_match_dqb1_high',
       'hla_nmdp_6', 'hla_match_c_low', 'hla_match_drb1_low',
       ...
       'tce_div_match_Unknown', 'donor_related_Related',
       'donor_related_Unknown', 'donor_related_Unrelated',
       'melphalan_dose_N_A__Mel_not_given', 'melphalan_dose_Unknown',
       'cardiac_Unknown', 'cardiac_Yes', 'pulm_moderate_Unknown',
       'pulm_moderate_Yes'],
      dtype='object', length=163)
X_test columns: Index(['ID', 'hla_match_c_high', 'hla_high_res_8', 'hla_low_res_6',
       'hla_high_res_6', 'hla_high_res_10', 'hla_match_dqb1_high',
       'hla_nmdp_6', 'hla_match_c_low', 'hla_match_drb1_low',
       'hla_match_dqb1_low', 'year_hct', 'hla_match_a_high', 'donor_age',
       'hla_match_b_low', 'age_at_hct', 'hla_match_a_low', 'hla_match_b_high',
       'comorbidity_score', 'karnofsky_score

In [9]:
print("X_train columns:", X_train.columns)
print("X_test columns:", X_test.columns)
print("X_train dtypes:", X_train.dtypes)
print("X_test dtypes:", X_test.dtypes)


X_train columns: Index(['ID', 'hla_match_c_high', 'hla_high_res_8', 'hla_low_res_6',
       'hla_high_res_6', 'hla_high_res_10', 'hla_match_dqb1_high',
       'hla_nmdp_6', 'hla_match_c_low', 'hla_match_drb1_low',
       ...
       'tce_div_match_Unknown', 'donor_related_Related',
       'donor_related_Unknown', 'donor_related_Unrelated',
       'melphalan_dose_N_A__Mel_not_given', 'melphalan_dose_Unknown',
       'cardiac_Unknown', 'cardiac_Yes', 'pulm_moderate_Unknown',
       'pulm_moderate_Yes'],
      dtype='object', length=163)
X_test columns: Index(['ID', 'hla_match_c_high', 'hla_high_res_8', 'hla_low_res_6',
       'hla_high_res_6', 'hla_high_res_10', 'hla_match_dqb1_high',
       'hla_nmdp_6', 'hla_match_c_low', 'hla_match_drb1_low',
       'hla_match_dqb1_low', 'year_hct', 'hla_match_a_high', 'donor_age',
       'hla_match_b_low', 'age_at_hct', 'hla_match_a_low', 'hla_match_b_high',
       'comorbidity_score', 'karnofsky_score', 'hla_low_res_8',
       'hla_match_drb1_high', 